In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
     sys.path.insert(0, project_root)

In [2]:
from litellm import completion
from dotenv import load_dotenv
from agent.tools.Weather import tools, get_current_weather
from agent.tools.utils import handle_tool_calls

load_dotenv()

history_messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris in celsius?"}]
model="gemini/gemini-2.5-flash-lite-preview-06-17"
model="ollama/qwen2.5vl:3b" # change to ollama/model qwen2.5vl:3b / qwen3:0.6b
api_base="http://localhost:11434" # add api base
available_functions = {"get_current_weather": get_current_weather}

first_response = completion(
  model=model,
  api_base="http://localhost:11434", # add api base
  messages=history_messages, 
  tools=tools,
  tool_choice="auto",
  )

In [3]:
import json
def detect_response_type(msg, history_msg, available_functions):
    # Append a dict representation of the message for JSON serialization
    history_msg.append({
        "role": msg.role,
        "content": msg.content,
        "tool_calls": getattr(msg, "tool_calls", None)
    })
    # Function call request
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        handle_tool_calls(msg.tool_calls, history_msg, available_functions)
        return "function_call"#, msg.tool_calls
    # Attempt structured JSON
    if msg.content:
        try:
            #parsed = json.loads(msg.content)
            return "structured_json"#, parsed
        except (json.JSONDecodeError, TypeError):
            return "plain_text"#, msg.content
    return "unknown"#, None

In [4]:
first_msg = first_response.choices[0].message
first_response_type = detect_response_type(first_msg, history_messages, available_functions)


Executing tool call
ChatCompletionMessageToolCall(function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), id='call_24671db5-fa94-44e6-bdbe-8291705f6290', type='function')
Result from tool call
{"location": "San Francisco, CA", "temperature": "30", "unit": "celsius"}



In [5]:
second_response = completion(
    model=model,
    messages=history_messages,
)

In [6]:
second_msg = second_response.choices[0].message
second_resp_type = detect_response_type(second_msg, history_messages, available_functions)
print(f"Detected: {second_resp_type}")
print(second_msg.content)

Detected: structured_json
### San Francisco, CA:
- **Temperature:** 30°C

### Tokyo, Japan:
- **Temperature:** 25°C

### Paris, France:
- **Temperature:** 20°C


In [7]:
print(json.dumps(history_messages, indent=2, default=str))

[
  {
    "role": "user",
    "content": "What's the weather like in San Francisco, Tokyo, and Paris in celsius?"
  },
  {
    "role": "assistant",
    "content": null,
    "tool_calls": [
      {
        "function": {
          "arguments": "{\"location\": \"San Francisco, CA\", \"unit\": \"celsius\"}",
          "name": "get_current_weather"
        },
        "id": "call_24671db5-fa94-44e6-bdbe-8291705f6290",
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_24671db5-fa94-44e6-bdbe-8291705f6290",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"San Francisco, CA\", \"temperature\": \"30\", \"unit\": \"celsius\"}"
  },
  {
    "role": "assistant",
    "content": "### San Francisco, CA:\n- **Temperature:** 30\u00b0C\n\n### Tokyo, Japan:\n- **Temperature:** 25\u00b0C\n\n### Paris, France:\n- **Temperature:** 20\u00b0C",
    "tool_calls": null
  }
]


# Image

In [8]:
response = completion(
    model = model, 
    messages=[
        {
            "role": "user",
            "content": [
                            {
                                "type": "text",
                                "text": "What’s in this image?"
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                                }
                            }
                        ]
        }
    ],
)
print(response)

ModelResponse(id='chatcmpl-682631b8-3524-470d-a4c2-3f5d45228fd3', created=1751768323, model='ollama/qwen2.5vl:3b', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='The image depicts a serene and picturesque landscape. It features a wooden boardwalk or pathway that meanders through a lush, green field. The pathway is constructed from wooden planks and is bordered by tall, green grasses. The field extends into the distance, creating a sense of depth and openness. In the background, there are trees and a clear blue sky with scattered, fluffy clouds. The overall scene is tranquil and inviting, suggesting a peaceful outdoor setting, possibly a nature reserve or a park.', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=101, prompt_tokens=1270, total_tokens=1371, completion_tokens_details=None, prompt_tokens_details=None))


In [9]:
response.choices[0].message.content


'The image depicts a serene and picturesque landscape. It features a wooden boardwalk or pathway that meanders through a lush, green field. The pathway is constructed from wooden planks and is bordered by tall, green grasses. The field extends into the distance, creating a sense of depth and openness. In the background, there are trees and a clear blue sky with scattered, fluffy clouds. The overall scene is tranquil and inviting, suggesting a peaceful outdoor setting, possibly a nature reserve or a park.'